In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [87.8 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-31 22:47:03--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-07-31 22:47:04 (11.3 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [6]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [7]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for total_votes count is greater than or equal to 20
vine20_df = vine_df.filter(col("total_votes")>=20)
vine20_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [8]:
helpful_vine_df = vine20_df.withColumn('percent_helpful', (vine20_df['helpful_votes']/vine20_df['total_votes']))
helpful_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|R3A1VQ3SQDXEJ3|          5|  

In [9]:
over_half_helpful_df = helpful_vine_df.filter(col("percent_helpful")>=0.5)
over_half_helpful_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|R3A1VQ3SQDXEJ3|          5|  

In [32]:
paid_df = over_half_helpful_df.filter(col("vine")=='Y')
paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|               0.9|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|               1.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|0.9285714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|0.9666666666666667|
| RD2BCTVS59A5L|          2|           20|         20|   Y|                N|               1.0|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|0.9615384615384616|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|              0.96|
|R25FI3J7WWOYXM|          3|  

In [33]:
unpaid_df = over_half_helpful_df.filter(col("vine")=='N')
unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|0.8709677419354839|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|0.9354838709677419|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|0.8333333333333334|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|0.9642857142857143|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|0.8372093023255814|
|R3A1VQ3SQDXEJ3|          5|  

In [34]:
paid_reviews = paid_df.count()
unpaid_reviews = unpaid_df.count()

In [35]:
total_reviews = over_half_helpful_df.count()
print(f"There are {total_reviews} reviews with at least 20 total votes and whose helpfulness is greater than or equal to 50%. {paid_reviews} of those are paid as part of the Vine program and {unpaid_reviews} are unpaid.")

There are 38010 reviews with at least 20 total votes and whose helpfulness is greater than or equal to 50%. 170 of those are paid as part of the Vine program and 37840 are unpaid.


In [36]:
five_star_reviews_df = over_half_helpful_df.filter(col("star_rating")==5)
five_star_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|               1.0|
| RPJLR6MFDPXXE|          5|           35|         36|   N|                Y|0.9722222222222222|
|R34X0ZGVB3YGYO|          5|           47|         47|   N|                Y|               1.0|
| RM6ZH5J3XDNHK|          5|           79|         87|   N|                Y|0.9080459770114943|
|R2E363K75EMW4K|          5|           66|         69|   N|                Y|0.9565217391304348|
| RLMZOQL11VHBX|          5|  

In [37]:
five_star_reviews = five_star_reviews_df.count()
print(f"There are {five_star_reviews} five-star reviews with at least 20 total votes and whose helpfulness is greater than or equal to 50%.")

There are 20677 five-star reviews with at least 20 total votes and whose helpfulness is greater than or equal to 50%.


In [38]:
five_star_paid_df = five_star_reviews_df.filter(col("vine")=='Y')
five_star_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|               1.0|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|0.9285714285714286|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|0.9666666666666667|
|R3C7MKX7MO6XUC|          5|          216|        219|   Y|                N|0.9863013698630136|
|R3FKBT77ZTUPLQ|          5|           15|         20|   Y|                N|              0.75|
|R3NSXBUR3O1CE8|          5|           21|         22|   Y|                N|0.9545454545454546|
|R1JORBU2J7HG3H|          5|           22|         25|   Y|                N|              0.88|
|R13X5H4F8GA5QP|          5|  

In [39]:
five_star_unpaid_df = five_star_reviews_df.filter(col("vine")=='N')
five_star_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|   percent_helpful|
+--------------+-----------+-------------+-----------+----+-----------------+------------------+
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|               1.0|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|           0.96875|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|               1.0|
| RPJLR6MFDPXXE|          5|           35|         36|   N|                Y|0.9722222222222222|
|R34X0ZGVB3YGYO|          5|           47|         47|   N|                Y|               1.0|
| RM6ZH5J3XDNHK|          5|           79|         87|   N|                Y|0.9080459770114943|
|R2E363K75EMW4K|          5|           66|         69|   N|                Y|0.9565217391304348|
| RLMZOQL11VHBX|          5|  

In [40]:
five_star_paid = five_star_paid_df.count()
print(f"There are {five_star_paid} five-star reviews that were paid for as part of the Vine program.")
five_star_unpaid = five_star_unpaid_df.count()
print(f"There are {five_star_unpaid} five-star reviews that were not part of the Vine program and therefore unpaid.")

There are 65 five-star reviews that were paid for as part of the Vine program.
There are 20612 five-star reviews that were not part of the Vine program and therefore unpaid.


In [42]:
five_star_paid_percent = five_star_paid / five_star_reviews * 100
five_star_unpaid_percent = five_star_unpaid / five_star_reviews *100

In [43]:
print(f"Of the {five_star_reviews} five-star reviews, {five_star_paid_percent}% were paid for through the Vine program and {five_star_unpaid_percent}% were unpaid.")

Of the 20677 five-star reviews, 0.31435894955747934% were paid for through the Vine program and 99.68564105044251% were unpaid.
